In [ ]:
# HSPd

## Last.fm Dataset

In [ ]:
import glob
from collections import defaultdict
import os.path

import pandas as pd

lfm_files = glob.glob('../../data/hit_song_prediction_lastfm/sample/*.pickle')


def merge_features(cols):
    cols = map(lambda c: '.'.join(str(c).split('.')[:2]), cols)
    cols = sorted(list(set(cols)))
    
    return cols
    
ess_lfm_features = None
for f in lfm_files:
    columns = list(filter(lambda c: not c.startswith('metadata') ,pd.read_pickle(f).columns))
    columns = merge_features(columns)
    
    if 'melspect' not in f:
        if ess_lfm_features:
            ess_lfm_features.intersection_update(set(columns))
        else:
            ess_lfm_features = set(columns)

ess_f = sorted(list(filter(lambda f: (f.startswith('highlevel') or f.startswith('lowlevel') or f.startswith('rhythm') or f.startswith('tonal')), ess_lfm_features)))

feat = defaultdict(list)
for f in ess_f:
    f = f.split('.')
    feat[f[0]].append(f[1])
    
display(dict(feat))

In [ ]:
import glob
import os.path

import pandas as pd

cleaned_files = glob.glob('../../data/hit_song_prediction_ismir2020/sample/*.pickle')
cleaned_files = list(filter(lambda f: 'cleaned' in f, cleaned_files))
    
ess_cleaned_features = None
for f in cleaned_files:
    columns = list(filter(lambda c: not str(c).startswith('metadata') ,pd.read_pickle(f).columns))
    columns = merge_features(columns)
    
    if 'melspect' not in f:
        if ess_cleaned_features:
            ess_cleaned_features.intersection_update(set(columns))
        else:
            ess_cleaned_features = set(columns)
            
display(ess_cleaned_features - ess_lfm_features)

In [ ]:
data = pd.read_parquet('../../data/hit_song_prediction_ismir2020/processed/msd_bb_mbid_cleaned_matches_ab_unique.parquet')

matches = pd.read_csv('../../data/hit_song_prediction_ismir2020/interim/msd_bb_mbid_cleaned_matches.csv')[['uuid', 'msd_id']]
non_matches = pd.read_csv('../../data/hit_song_prediction_ismir2020/interim/msd_bb_mbid_non_matches.csv')

In [ ]:
mapping = pd.concat([matches[['uuid', 'msd_id']], non_matches[['uuid', 'msd_id']]]).drop_duplicates()

msd_year = pd.read_csv('../../data/millionsongdataset/additional_files/tracks_per_year.txt', sep='<SEP>', names=['year', 'msd_id', 'artist', 'title'])[['year', 'msd_id']]

mapping_year = mapping.merge(msd_year, on=['msd_id'])

In [ ]:
import numpy as np

uuid_year = mapping_year[['uuid', 'year']].groupby(by='uuid').agg(lambda x : x.max() == x.min())
uuid_year = uuid_year[uuid_year['year']].reset_index()[['uuid']]

display(uuid_year.shape, uuid_year[['uuid']].drop_duplicates().shape)


cor_year = uuid_year[['uuid']].drop_duplicates().merge(mapping_year[['uuid', 'year']].drop_duplicates(), on=['uuid'])

In [ ]:
data.shape, data.merge(cor_year, on=['uuid']).shape

In [ ]:
some = mapping_year[['uuid', 'year']].groupby(by='uuid').agg([lambda x : x.max() == x.min(), np.min, np.max])
some[some[('year', '<lambda_0>')] == 0]

In [ ]:
data_lfm = pd.read_parquet('../../data/hit_song_prediction_lastfm/processed/msd_lastfm_matches_ab_unique.parquet')
matches_lfm = pd.read_csv('../../data/hit_song_prediction_lastfm/interim/msd_lastfm_matches.csv')[['uuid', 'msd_id']].drop_duplicates()

mapping_year_lfm = matches_lfm.merge(msd_year, on=['msd_id'])

uuid_year_lfm = mapping_year_lfm[['uuid', 'year']].groupby(by='uuid').agg(lambda x : x.max() == x.min())
uuid_year_lfm = uuid_year_lfm[uuid_year_lfm['year']].reset_index()[['uuid']]

display(uuid_year_lfm.shape, uuid_year_lfm.drop_duplicates().shape)


cor_year_lfm = uuid_year_lfm[['uuid']].drop_duplicates().merge(mapping_year_lfm[['uuid', 'year']].drop_duplicates(), on=['uuid'])

In [ ]:
data_lfm.shape, data_lfm.merge(cor_year_lfm, on=['uuid']).shape

In [ ]:
data_lfm.shape, data_lfm['uuid'].drop_duplicates().shape, cor_year_lfm.shape, cor_year_lfm.drop_duplicates().shape

In [ ]:
data_lfm[['lastfm_playcount', 'lastfm_listener_count']], data[['lastfm_playcount', 'lastfm_listener_count']]